# 3.0 - Comparación de Modelos: Datos Básicos vs. Datos Enriquecidos

**Objetivo:** Este notebook tiene como finalidad comparar el rendimiento de un modelo de Machine Learning entrenado con dos conjuntos de datos diferentes:

1.  **Modelo 1 (Básico):** Utiliza `karcal_data_cleaned_raw.csv`, que contiene los datos limpios pero sin las características adicionales de estado, historial, multas, etc.
2.  **Modelo 2 (Enriquecido):** Utiliza `karcal_data_cleaned.csv`, el cual fue procesado con el script `clean_data.py` y contiene todas las características nuevas.

El objetivo es medir con métricas como el **Error Absoluto Medio (MAE)** y el **Coeficiente de Determinación (R²)** si el proceso de enriquecimiento de datos resultó en un modelo más preciso para predecir la `oferta_ganadora`.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, r2_score

# --- Configuración de Visualización ---
pd.options.display.float_format = lambda x: f'{x:,.0f}'.replace(',', '.')
%matplotlib inline
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

## Modelo 1: Datos Limpios Básicos (sin enriquecimiento)

Primero, cargamos y entrenamos el modelo usando el archivo CSV con la limpieza básica, que no incluye las columnas de estado legal, condición física, etc.

In [2]:
# --- Cargar Datos Limpios Básicos ---
file_path_raw = '../data/clean/karcal_data_cleaned_raw.csv'
try:
    df_raw = pd.read_csv(file_path_raw)
    print(f"Datos básicos cargados desde: {file_path_raw}")
except FileNotFoundError:
    print(f"Error: No se encontró el archivo {file_path_raw}. Asegúrate de que exista.")

# --- Preparación del Modelo 1 ---
features_raw = [
    'antiguedad', 'kilometraje', 'cilindrada', 'visitas', 
    'numero_pujas', 'valor_inicial', 'marca', 'transmisión', 'combustible'
]
target = 'oferta_ganadora'

df_model_raw = df_raw.dropna(subset=features_raw + [target])
X_raw = df_model_raw[features_raw]
y_raw = df_model_raw[target]

X_train_raw, X_test_raw, y_train_raw, y_test_raw = train_test_split(X_raw, y_raw, test_size=0.2, random_state=42)

# Preprocesador para el modelo básico
numeric_features_raw = X_raw.select_dtypes(include=np.number).columns.tolist()
categorical_features_raw = X_raw.select_dtypes(include=['object']).columns.tolist()

preprocessor_raw = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numeric_features_raw),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features_raw)
    ])

# Pipeline del Modelo 1
model_raw = Pipeline(steps=[
    ('preprocessor', preprocessor_raw),
    ('regressor', RandomForestRegressor(n_estimators=100, random_state=42))
])

# Entrenamiento y Evaluación del Modelo 1
print("\nEntrenando Modelo 1 (Básico)...")
model_raw.fit(X_train_raw, y_train_raw)
y_pred_raw = model_raw.predict(X_test_raw)

mae_raw = mean_absolute_error(y_test_raw, y_pred_raw)
r2_raw = r2_score(y_test_raw, y_pred_raw)

print("--- Resultados del Modelo 1 (Básico) ---")
print(f"Error Absoluto Medio (MAE): ${mae_raw:,.0f}")
print(f"Coeficiente de Determinación (R²): {r2_raw:.4f}")

Datos básicos cargados desde: ../data/clean/karcal_data_cleaned_raw.csv

Entrenando Modelo 1 (Básico)...
--- Resultados del Modelo 1 (Básico) ---
Error Absoluto Medio (MAE): $1,316,000
Coeficiente de Determinación (R²): 0.6737


## Modelo 2: Datos Limpios Enriquecidos

Ahora, repetimos el proceso exactamente igual, pero utilizando el archivo CSV que contiene todas las características adicionales generadas por el script de limpieza.

In [3]:
# --- Cargar Datos Limpios Enriquecidos ---
file_path_enriched = '../data/clean/karcal_data_cleaned.csv'
try:
    df_enriched = pd.read_csv(file_path_enriched)
    print(f"Datos enriquecidos cargados desde: {file_path_enriched}")
except FileNotFoundError:
    print(f"Error: No se encontró el archivo {file_path_enriched}.")

# --- Preparación del Modelo 2 ---
features_enriched = [
    'antiguedad', 'kilometraje', 'cilindrada', 'visitas', 'numero_pujas',
    'valor_inicial', 'numero_propietarios', 'meses_dueño_actual',
    'funciona', 'tiene_llaves', 'tiene_multas', 'monto_multas_utm',
    'marca', 'transmisión', 'combustible', 'tracción', 'mandante'
]
# Asegurarnos de que todas las columnas existan en el DataFrame
features_enriched = [f for f in features_enriched if f in df_enriched.columns]


df_model_enriched = df_enriched.dropna(subset=features_enriched + [target])
X_enriched = df_model_enriched[features_enriched]
y_enriched = df_model_enriched[target]

X_train_enriched, X_test_enriched, y_train_enriched, y_test_enriched = train_test_split(X_enriched, y_enriched, test_size=0.2, random_state=42)

# Preprocesador para el modelo enriquecido
numeric_features_enriched = X_enriched.select_dtypes(include=np.number).columns.tolist()
categorical_features_enriched = X_enriched.select_dtypes(include=['object']).columns.tolist()

preprocessor_enriched = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numeric_features_enriched),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features_enriched)
    ],
    remainder='passthrough'
)

# Pipeline del Modelo 2
model_enriched = Pipeline(steps=[
    ('preprocessor', preprocessor_enriched),
    ('regressor', RandomForestRegressor(n_estimators=100, random_state=42))
])

# Entrenamiento y Evaluación del Modelo 2
print("\nEntrenando Modelo 2 (Enriquecido)...")
model_enriched.fit(X_train_enriched, y_train_enriched)
y_pred_enriched = model_enriched.predict(X_test_enriched)

mae_enriched = mean_absolute_error(y_test_enriched, y_pred_enriched)
r2_enriched = r2_score(y_test_enriched, y_pred_enriched)

print("--- Resultados del Modelo 2 (Enriquecido) ---")
print(f"Error Absoluto Medio (MAE): ${mae_enriched:,.0f}")
print(f"Coeficiente de Determinación (R²): {r2_enriched:.4f}")

Datos enriquecidos cargados desde: ../data/clean/karcal_data_cleaned.csv

Entrenando Modelo 2 (Enriquecido)...
--- Resultados del Modelo 2 (Enriquecido) ---
Error Absoluto Medio (MAE): $561
Coeficiente de Determinación (R²): 1.0000


## 3. Comparación de Resultados y Conclusión

In [4]:
# --- Crear un DataFrame para la comparación ---
results_df = pd.DataFrame({
    'Métrica': ['Error Absoluto Medio (MAE)', 'Coeficiente de Determinación (R²)'],
    'Modelo Básico (raw)': [f"${mae_raw:,.0f}", f"{r2_raw:.4f}"],
    'Modelo Enriquecido': [f"${mae_enriched:,.0f}", f"{r2_enriched:.4f}"]
})

print("--- Tabla Comparativa de Rendimiento ---")
display(results_df.set_index('Métrica'))

--- Tabla Comparativa de Rendimiento ---


,Modelo Básico (raw),Modelo Enriquecido
Métrica,,
Error Absoluto Medio (MAE),"$1,316,000",$561
Coeficiente de Determinación (R²),0.6737,1.0000


### Conclusión

Al analizar la tabla comparativa, se puede observar claramente el impacto del enriquecimiento de datos.

* **Error Absoluto Medio (MAE):** El MAE del modelo enriquecido es **[menor/mayor]** que el del modelo básico. Esto significa que, en promedio, las predicciones del nuevo modelo se equivocan en [resta de los MAE] pesos **[menos/más]**. Una reducción del MAE indica una mejora en la precisión.

* **Coeficiente de Determinación (R²):** El R² del modelo enriquecido es **[mayor/menor]**. Un R² más cercano a 1 indica que el modelo es capaz de explicar un porcentaje **[mayor/menor]** de la variabilidad del precio de la oferta ganadora.

**En resumen, la inclusión de características como el estado funcional del vehículo, su historial de propiedad y la información sobre multas ha [mejorado significativamente / tenido un impacto moderado / no ha mejorado] la capacidad predictiva del modelo.** Esto demuestra el valor de un proceso de limpieza y feature engineering más profundo.